In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import json
import os

In [2]:
# A function for reading an image from a path and returning its features (keypoints and descriptors)
def extract_features(image_path):

    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)

    return keypoints, descriptors

In [158]:
def match_features(descriptors_list):
    # Brute force matcher
    bf = cv2.BFMatcher()

    # Dictionary to store matches between all pairs of images
    matches_dict = {}

    # Iterate over all pairs of images
    for i in range(len(descriptors_list)):
        print("Looping through image: ", i)
        for j in range(len(descriptors_list)):
            # skip if the image is itself
            if i == j: continue
            # Match descriptors between image i and image j
            matches = bf.knnMatch(descriptors_list[i], descriptors_list[j], k=2)
            # only consider good matches
            threshold=0.5
            good_matches = []
            for m, n in matches:
                if m.distance < threshold * n.distance:
                    good_matches.append(m)
            matches_dict[(i, j)] = good_matches

    return matches_dict

# this function is for getting all the matches with a single image as the source (only used for testing)
def single_image_match_features(descriptors_list, i=0):
    bf = cv2.BFMatcher()
    matches_dict = {}

    for j in range(len(descriptors_list)):
        if i == j: continue
        # Match descriptors between a single image j and image i
        matches = bf.knnMatch(descriptors_list[i], descriptors_list[j], k=2)

        threshold=0.7
        good_matches = []
        for m, n in matches:
            if m.distance < threshold * n.distance:
                good_matches.append(m)
        matches_dict[(i, j)] = good_matches

    return matches_dict


In [123]:
# List of image paths
image_paths = [f for f in os.listdir('assets/airpods/test/') if '.JPEG' in f]

# Extracter features and descriptors for the images and store them in lists using the image index
descriptors_list = []
keypoints_list = []
for image_path in image_paths:
    keypoints, descriptors = extract_features('assets/airpods/test/' + image_path)
    descriptors_list.append(descriptors)
    keypoints_list.append(keypoints)

In [159]:
# Get feature matches for all the images
matches_dict = match_features(descriptors_list)
# single_matches_dict = single_image_match_features(descriptors_list, i=0) # this function was for testing

Looping through image:  0
Looping through image:  1
Looping through image:  2
Looping through image:  3
Looping through image:  4
Looping through image:  5
Looping through image:  6
Looping through image:  7
Looping through image:  8
Looping through image:  9
Looping through image:  10
Looping through image:  11
Looping through image:  12
Looping through image:  13
Looping through image:  14
Looping through image:  15
Looping through image:  16
Looping through image:  17
Looping through image:  18
Looping through image:  19
Looping through image:  20
Looping through image:  21
Looping through image:  22
Looping through image:  23
Looping through image:  24
Looping through image:  25
Looping through image:  26
Looping through image:  27
Looping through image:  28
Looping through image:  29
Looping through image:  30
Looping through image:  31
Looping through image:  32
Looping through image:  33
Looping through image:  34
Looping through image:  35
Looping through image:  36
Looping thr

In [314]:
# Find the best source image (where the transformation matrices will be built from)
# Based on what fraction of the remaining images that share at least 8 good matches with the source image
# The best % is chosen as the source image
best_sources = {}
for key in matches_dict:
    source = key[0]
    if len(matches_dict[key]) >= 8:
        if best_sources.get(source):
            best_sources[source] += 1
        else:
            best_sources[source] = 1

best_source = sorted(best_sources.items(), key=lambda item: item[1])[-1]
# for key in best_sources:
#     print(key, best_sources[key]) # uncomment if you want to see how many values are in each set of good matches
print("The best source image is: ", best_source)

0 22
1 15
2 23
3 21
4 11
5 26
6 23
7 24
8 14
9 12
10 16
11 5
12 2
13 1
15 2
16 21
17 25
18 27
19 15
20 3
21 20
22 3
23 21
24 24
25 22
26 23
27 22
28 15
29 10
30 5
31 3
32 6
33 4
34 5
35 10
36 5
37 3
38 8
39 11
40 4
41 6
42 10
43 11
44 16
45 8
46 7
47 8
48 10
49 8
50 4
51 9
52 19
53 22
55 7
56 2
57 1
58 22
59 13
60 9
61 10
62 6
The best source image is:  (18, 27)


In [320]:
# Prune images that fail to have more 8 or more good matches
pruned_match_dict = {}
for i in range(len(image_paths)):
    if i == best_source[0]: continue
    if len(matches_dict[(best_source[0], i)]) >= 8:
        pruned_match_dict[(best_source[0], i)] = matches_dict[(best_source[0], i)]


In [307]:
# We got the focal length from the calibration for intrinsic parameters
focal_length = 1.55197091e+03
# We are assuming the focal point is the centre of the image - the image is 1773x1773 -> (1773/2, 1773/2) 
principal_point = (886.5, 886.5)
# Building the intrinsic matrix for this specific camera
K = np.array([[focal_length, 0, principal_point[0]],
              [0, focal_length, principal_point[1]],
              [0, 0, 1]])


In [376]:
def flip_y_axes(T):
    R_flip = np.array([[1, 0, 0],
                       [0, -1, 0],
                       [0, 0, 1]])
    
    # Applying the rotation to the 3x3 top-left submatrix of T
    T[:3, :3] = T[:3, :3] @ R_flip
    return T

In [379]:
def estimate_relative_pose(keypoints1, keypoints2, matches, camera_matrix):
    if len(matches) < 8: return None, None
    points1 = np.float32([keypoints1[m.queryIdx].pt for m in matches])
    points2 = np.float32([keypoints2[m.trainIdx].pt for m in matches])

    E, _ = cv2.findEssentialMat(points1, points2, camera_matrix, method=cv2.RANSAC, prob=0.999, threshold=1.0)
    _, R, t, _ = cv2.recoverPose(E, points1, points2, camera_matrix)

    return R, t


# Initialize array to hold transformation matrices
num_images = len(pruned_match_dict)
transformation_matrices = {}
transformation_matrices[best_source[0]] = flip_y_axes(np.identity(4))

for (idx1, idx2), matches in pruned_match_dict.items():
    R, t = estimate_relative_pose(keypoints_list[idx1], keypoints_list[idx2], matches, K)
    if R is None: continue
    transformation_matrix = flip_y_axes(np.eye(4))
    transformation_matrix[:3, :3] = R
    transformation_matrix[:3, 3] = t[:, 0]
    transformation_matrices[idx2] = flip_y_axes(np.linalg.inv(transformation_matrix))

In [380]:
# Rewrite transformation file generated by InstantNGP to test results
file = open('assets/airpods/test/transforms.json')
data = json.load(file)

with open('assets/airpods/test/transforms4.json', 'w') as f:
    data["aabb_scale"] = 8
    for i in range(len(data['frames'])):
        file_path = data['frames'][i]['file_path']
        img_index = int(file_path[4:8]) - 72
        
        if transformation_matrices.get(img_index) is not None:
            data['frames'][i]['transform_matrix'] = transformation_matrices[img_index].tolist()
        else:
            print("pruned image:", img_index + 72)
            data['frames'][i] = None

    json.dump(data, f)

pruned image: 83
pruned image: 84
pruned image: 80
pruned image: 87
pruned image: 86
pruned image: 92
pruned image: 94
pruned image: 103
pruned image: 105
pruned image: 102
pruned image: 111
pruned image: 112
pruned image: 104
pruned image: 110
pruned image: 113
pruned image: 107
pruned image: 106
pruned image: 108
pruned image: 114
pruned image: 117
pruned image: 118
pruned image: 109
pruned image: 119
pruned image: 115
pruned image: 116
pruned image: 122
pruned image: 129
pruned image: 120
pruned image: 121
pruned image: 131
pruned image: 133
pruned image: 134


In [335]:
# This block of code was used to visualize the quality of the matches to see the value of pruning
def draw(img1_path, kp1, img2_path, kp2, good_matches):
    img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)
    # Draw matches
    img_matches = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    # Display the matches
    cv2.namedWindow("Sift_Matches", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Sift_Matches", 2000, 1000)
    cv2.imshow("Sift_Matches", img_matches)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Pre-pruning on best_source
# source = best_source[0]
# for i in range(len(image_paths)):
#     if i == source: continue
#     draw('assets/airpods/test/' + image_paths[source], keypoints_list[source], 'assets/airpods/test/' + image_paths[i], keypoints_list[i], matches_dict[(source, i)])

# Post pruning
print(len(pruned_match_dict))
for (source, i), matches in pruned_match_dict.items():
    print(len(matches))
    draw('assets/airpods/test/' + image_paths[source], keypoints_list[source], 'assets/airpods/test/' + image_paths[i], keypoints_list[i], matches)

27
17
12
22
30
10
37
86
56
8
40
10
22
100
33
23
35
61
38
46
22
33
27
8
22
28
26
9
